In [6]:
import pandas as pd
import data.get_dataset as ds
import numpy as np

In [7]:
class Preprocessor:
    def __init__(self):
        self.dataset = ds.get_dataset()
        self.globmap = {}

    def setMap(self):
        self.dictMap = dict(tuple(self.dataset.groupby('Symbol')))
        print(self.dictMap)

In [8]:
p = Preprocessor()
p.setMap()

{'AAPL':       Symbol        Date        Open        High         Low       Close  \
23217   AAPL  2017-09-26   37.945000   38.480000   37.922501   38.285000   
23218   AAPL  2017-09-27   38.450001   38.680000   38.384998   38.557499   
23219   AAPL  2017-09-28   38.472500   38.570000   38.174999   38.320000   
23220   AAPL  2017-09-29   38.302502   38.532501   38.000000   38.529999   
23221   AAPL  2017-10-02   38.564999   38.612499   38.180000   38.452499   
...      ...         ...         ...         ...         ...         ...   
24470   AAPL  2022-09-19  149.309998  154.559998  149.100006  154.479996   
24471   AAPL  2022-09-20  153.399994  158.080002  153.080002  156.899994   
24472   AAPL  2022-09-21  157.339996  158.740005  153.600006  153.720001   
24473   AAPL  2022-09-22  152.380005  154.470001  150.910004  152.740005   
24474   AAPL  2022-09-23  151.190002  151.470001  148.559998  150.429993   

        Adj Close     Volume  
23217   36.334969  146640000  
23218   36.59358

In [144]:
def close_diff(symbol):
    out = []
    n = len(p.dictMap[symbol]['Open'])
    if not 'CLOSE_OPEN_DIFF' in p.dictMap[symbol]:
        p.dictMap[symbol]['CLOSE_OPEN_DIFF'] = p.dictMap[symbol]['Close'] - p.dictMap[symbol]['Open']
    if not 'CLOSE_DIFF' in p.dictMap[symbol]:
        val = [None]
        for i in range(1, n):
            val.append(p.dictMap[symbol]['Close'].iloc[i] - p.dictMap[symbol]['Close'].iloc[i-1])
        p.dictMap[symbol]['CLOSE_DIFF'] = val
    print(p.dictMap[symbol][['Open', 'Close', 'CLOSE_OPEN_DIFF', 'CLOSE_DIFF']])

In [145]:
def relative_strength_index(symbol, period):
    startIndex = period + 1
    if not 'CLOSE_DIFF' in p.dictMap[symbol]:
        close_diff(symbol)
    val = p.dictMap[symbol]['CLOSE_DIFF']

    avg_gain = 0
    avg_loss = 0
    n = len(val)

    for i in range(1, period + 1):
        num = val.iloc[i]
        if num > 0:
            avg_gain += num
        else:
            avg_loss += -num
    
    avg_gain /= period
    avg_loss /= period

    rsi = [100 - 100/(1+(avg_gain/avg_loss))]
    for i in range(period + 1, n):
        num = val.iloc[i]
        if num > 0:
            curr_gain = num
            curr_loss = 0
        else:
            curr_gain = 0
            curr_loss = -num
        avg_gain = (avg_gain * (period - 1) + curr_gain)/period
        avg_loss = (avg_loss * (period - 1) + curr_loss)/period
        rsi.append(100 - 100/(1+(avg_gain/avg_loss)))
    return np.array(rsi)
        

In [1]:
# print(p.dictMap)
relative_strength_index('AAPL', 14)


NameError: name 'relative_strength_index' is not defined

In [147]:
def high_low(symbol, period):
    dictMap = p.dictMap[symbol]
    highest_highs = []
    lowest_lows = []
    n = len(dictMap['High'])
    for i in range(n-period):
        highest_high = dictMap['High'].iloc[i]
        lowest_low = dictMap['Low'].iloc[i]
        for j in range(1, period):
            highest_high = max(dictMap['High'].iloc[i+j], highest_high)
            lowest_low = min(dictMap['Low'].iloc[i+j], lowest_low)
        # print(highest_high)
        # print(lowest_low)
        highest_highs.append(highest_high)
        lowest_lows.append(lowest_low)
    return (highest_highs, lowest_lows, n)


In [148]:
def stochastic(symbol, period):
    hh, ll, n = high_low(symbol, period)
    k = []
    for i in range(n-period):
        k.append(100*(p.dictMap[symbol]['Close'].iloc[i+period-1] - ll[i])/(hh[i] - ll[i]))
    return np.array(k)

In [149]:
stochastic('AAPL', 14)

array([83.9937141 , 98.50005   , 95.4904602 , ..., 53.68155444,
       33.66901196, 27.50163625])

In [150]:
def typicalPrice(symbol):
    out = []
    n = len(p.dictMap[symbol]['Open'])
    if not 'TYPICAL_PRICE' in p.dictMap[symbol]:
        p.dictMap[symbol]['TYPICAL_PRICE'] = (p.dictMap[symbol]['Close'] + p.dictMap[symbol]['High'] + p.dictMap[symbol]['Low']) / 3
    print(p.dictMap[symbol][['High', 'Low', 'Close', 'TYPICAL_PRICE']])

In [151]:
def commodityChannelIndex(symbol, period):
    n = len(p.dictMap[symbol]['Open'])
    cci = []
    typicalPrice(symbol)
    for i in range(n-period):
        sma = sum(p.dictMap[symbol]['TYPICAL_PRICE'].iloc[i:i+period]) / period
        md = sum(abs(p.dictMap[symbol]['TYPICAL_PRICE'].iloc[i:i+period] - sma)) / period
        cci.append((p.dictMap[symbol]['TYPICAL_PRICE'].iloc[i+period-1] - sma) / (0.015*md))
    return np.array(cci)

In [152]:
commodityChannelIndex('AAPL', 40)

             High         Low       Close  TYPICAL_PRICE    MONEY_FLOW
23217   38.480000   37.922501   38.285000      38.229167  5.605925e+09
23218   38.680000   38.384998   38.557499      38.540832  3.931812e+09
23219   38.570000   38.174999   38.320000      38.355000  3.376084e+09
23220   38.532501   38.000000   38.529999      38.354167  4.034828e+09
23221   38.612499   38.180000   38.452499      38.414999  2.873258e+09
...           ...         ...         ...            ...           ...
24470  154.559998  149.100006  154.479996     152.713333  1.244220e+10
24471  158.080002  153.080002  156.899994     156.019999  1.680176e+10
24472  158.740005  153.600006  153.720001     155.353337  1.579894e+10
24473  154.470001  150.910004  152.740005     152.706670  1.323241e+10
24474  151.470001  148.559998  150.429993     150.153331  1.440559e+10

[1258 rows x 5 columns]


array([  70.8205207 ,   88.05569794,   98.04198489, ...,  -68.86908248,
        -78.38188375, -106.37769793])

In [153]:
def movingAverageConvergenceDivergence(symbol):
    n = len(p.dictMap[symbol]['Open'])
    ema_12 = sum(p.dictMap[symbol]['Close'].iloc[:12]) / 12
    K = 2/(12+1)
    ema12 = [ema_12]
    for i in range(12, n):
        ema_12 = (p.dictMap[symbol]['Close'].iloc[i] - ema_12) * K + ema_12
        ema12.append(ema_12)
    
    K = 2/(26+1)
    ema_26 = sum(p.dictMap[symbol]['Close'].iloc[:26]) / 26
    ema26 = [ema_26]
    for i in range(26, n):
        ema_26 = (p.dictMap[symbol]['Close'].iloc[i] - ema_26) * K + ema_26
        ema26.append(ema_26)
    macd = np.array(ema12[14:]) - np.array(ema26)

    sig = sum(macd[:9]) / 9
    K = 0.2
    signal = [sig]
    for i in range(9, len(macd)):
        sig = (macd[i]-sig) * K + sig
        signal.append(sig)

    return (macd, np.array(signal))




In [154]:
movingAverageConvergenceDivergence('AAPL')

(array([ 0.87362954,  0.93281804,  0.99289087, ..., -2.17281566,
        -2.25752139, -2.482434  ]),
 array([ 1.19790437,  1.2400001 ,  1.25431503, ..., -1.7049278 ,
        -1.81544652, -1.94884402]))

In [164]:
def moneyFlowHelper(symbol):
    out = []
    n = len(p.dictMap[symbol]['Open'])
    if not 'TYPICAL_PRICE' in p.dictMap[symbol]:
        typicalPrice(symbol)
    if not 'MONEY_FLOW' in p.dictMap[symbol]:
        p.dictMap[symbol]['MONEY_FLOW'] = p.dictMap[symbol]['TYPICAL_PRICE'] * p.dictMap[symbol]['Volume']
    moneyFlowDiff = [None]
    for i in range(1, n):
        moneyFlowDiff.append(p.dictMap[symbol]['MONEY_FLOW'].iloc[i] - p.dictMap[symbol]['MONEY_FLOW'].iloc[i-1])
    p.dictMap[symbol]['MONEY_FLOW_DIFF'] = moneyFlowDiff
    print(p.dictMap[symbol][['High', 'Low', 'Close', 'MONEY_FLOW', 'MONEY_FLOW_DIFF']])

In [167]:
def moneyFlow(symbol, period):
    typicalPrice(symbol)
    n = len(p.dictMap[symbol]['Open'])
    moneyFlowHelper(symbol)
    mfi = []
    for i in range(1, n-period):
        diff = np.array(p.dictMap[symbol]['MONEY_FLOW_DIFF'].iloc[i:i+period])
        flow = np.array(p.dictMap[symbol]['MONEY_FLOW'].iloc[i:i+period])
        pmf = 0
        nmf = 0
        for j in range(period):
            if diff[j] < 0:
                nmf += flow[j]
            else:
                pmf += flow[j]
        ratio = pmf / nmf
        mfi.append(100 - (100 / (1 + ratio)))
    return np.array(mfi)

In [168]:
moneyFlow('AAPL', 14)

             High         Low       Close  TYPICAL_PRICE    MONEY_FLOW
23217   38.480000   37.922501   38.285000      38.229167  5.605925e+09
23218   38.680000   38.384998   38.557499      38.540832  3.931812e+09
23219   38.570000   38.174999   38.320000      38.355000  3.376084e+09
23220   38.532501   38.000000   38.529999      38.354167  4.034828e+09
23221   38.612499   38.180000   38.452499      38.414999  2.873258e+09
...           ...         ...         ...            ...           ...
24470  154.559998  149.100006  154.479996     152.713333  1.244220e+10
24471  158.080002  153.080002  156.899994     156.019999  1.680176e+10
24472  158.740005  153.600006  153.720001     155.353337  1.579894e+10
24473  154.470001  150.910004  152.740005     152.706670  1.323241e+10
24474  151.470001  148.559998  150.429993     150.153331  1.440559e+10

[1258 rows x 5 columns]
             High         Low       Close    MONEY_FLOW  MONEY_FLOW_DIFF
23217   38.480000   37.922501   38.285000  5.60592

array([45.97645144, 46.961414  , 47.83300446, ..., 64.18068546,
       56.82170911, 56.38915404])

In [175]:
def accDist(symbol):
    n = len(p.dictMap[symbol]['High'])
    mfMultiplier = ((p.dictMap[symbol]['Close'] - p.dictMap[symbol]['Low']) - (p.dictMap[symbol]['High'] - p.dictMap[symbol]['Close']))/ (p.dictMap[symbol]['High'] - p.dictMap[symbol]['Low'])
    mfv = mfMultiplier * p.dictMap[symbol]['Volume']
    acc = [mfv.iloc[0]]
    for i in range(1, n):
        acc.append(acc[i-1] + mfv.iloc[i])
    return np.array(acc)

In [176]:
accDist('AAPL')

array([4.40575738e+07, 6.13484397e+07, 3.79504686e+07, ...,
       6.57954919e+09, 6.58198338e+09, 6.60934702e+09])